In [108]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math

In [78]:
from sklearn.preprocessing import OneHotEncoder

In [67]:
UNK = "$UNK$"
NUM = "$NUM$"
NONE = "O"

BEGIN = "<s>"
END = "</s>"

In [58]:
def load_vocab(filename):
    """Loads vocab from a file

    Args:
        filename: (string) the format of the file must be one word per line.

    Returns:
        d: dict[word] = index

    """
    try:
        d = dict()
        with open(filename) as f:
            for idx, word in enumerate(f):
                word = word.strip()
                d[word] = idx

    except IOError:
        raise MyIOError(filename)
    return d

In [68]:
def get_processing_word(vocab_words=None, vocab_chars=None,
                    lowercase=False, chars=False, allow_unk=True):
    """Return lambda function that transform a word (string) into list,
    or tuple of (list, id) of int corresponding to the ids of the word and
    its corresponding characters.

    Args:
        vocab: dict[word] = idx

    Returns:
        f("cat") = ([12, 4, 32], 12345)
                 = (list of char ids, word id)

    """
    def f(word):
        # 0. get chars of words
        if vocab_chars is not None and chars == True:
            char_ids = []
            for char in word:
                # ignore chars out of vocabulary
                if char in vocab_chars:
                    char_ids += [vocab_chars[char]]

        # 1. preprocess word
        if lowercase:
            word = word.lower()
        if word.isdigit():
            word = NUM

        # 2. get id of word
        if vocab_words is not None:
            if word in vocab_words:
                word = vocab_words[word]
            else:
                if allow_unk:
                    word = vocab_words[UNK]
                else:
                    raise Exception("Unknow key is not allowed. Check that "\
                                    "your vocab (tags?) is correct")

        # 3. return tuple char ids, word id
        if vocab_chars is not None and chars == True:
            return char_ids, word
        else:
            return word

    return f


In [49]:
PATH_TO_TRAIN = "../../data/train.tsv"

In [50]:
PATH_TO_TEST = "../../data/public.tsv"

In [59]:
PATH_TO_VOCAB = "../../data/vocab.txt"

In [51]:
train_column_names = ['context_id', 'context_2', 'context_1', 
                           'context_0', 'reply_id', 'reply', 'label', 'confidence']

In [148]:
test_column_names = ['context_id', 'context_2', 'context_1', 
                           'context_0', 'reply_id', 'reply']

In [183]:
train = pd.read_csv(PATH_TO_TRAIN, error_bad_lines=False, sep = '[  . ? , !]?\t', 
                   header=None)

/Users/kitashov/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [184]:
train.head()

,0,1,2,3,4,5,6,7
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",0,не могу,good,0.875352
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",1,"нет , звонить буду я",neutral,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",2,"слушай , я не мог уйти",bad,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",3,я не прекращу звонить,good,0.982530
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",4,я звоню им,good,0.838054


In [185]:
train.columns = train_column_names

In [186]:
train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",0,не могу,good,0.875352
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",1,"нет , звонить буду я",neutral,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",2,"слушай , я не мог уйти",bad,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",3,я не прекращу звонить,good,0.982530
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",4,я звоню им,good,0.838054


# Sentences to indices

In [187]:
vocab = load_vocab(PATH_TO_VOCAB)

In [188]:
def sentence_to_indices(sentence, preprocess_word=get_processing_word(vocab_words=vocab)):
    try:
        sentence = [preprocess_word(word) for word in sentence.split()]
    except AttributeError:
        sentence = [preprocess_word(str(word))]
    return sentence

In [191]:
sentence_to_indeces('кстати')

[773239]

In [181]:
train['context_2'] = train['context_2'].apply(lambda x: sentence_to_indices(x))

In [182]:
train['context_1'] = train['context_1'].apply(lambda x: sentence_to_indices(x))

In [157]:
train['context_0'] = train['context_0'].apply(lambda x: sentence_to_indices(x))

In [158]:
train['reply'] = train['reply'].apply(lambda x: sentence_to_indices(x))

In [159]:
train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",0,"[743031, 1101989]",good,0.875352
1,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",1,"[1141967, 1463197, 1431000, 1100672, 334254]",neutral,0.900968
2,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",2,"[1448996, 1463197, 334254, 743031, 1688119, 15...",bad,0.884320
3,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",3,"[334254, 743031, 1103971, 1431000]",good,0.982530
4,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",4,"[334254, 355899, 1430584]",good,0.838054


## One-hot encoding

In [160]:
d = {"good": [0, 0, 1], "neutral": [0, 1, 0], "bad": [1, 0, 0]}

In [161]:
train.label = train.label.apply(lambda x: d[x])

In [162]:
train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",0,"[743031, 1101989]","[0, 0, 1]",0.875352
1,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",1,"[1141967, 1463197, 1431000, 1100672, 334254]","[0, 1, 0]",0.900968
2,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",2,"[1448996, 1463197, 334254, 743031, 1688119, 15...","[1, 0, 0]",0.884320
3,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",3,"[334254, 743031, 1103971, 1431000]","[0, 0, 1]",0.982530
4,22579918886,"[654394, 1561605, 16734, 1554817, 306154, 1561...","[1830003, 1463197, 334254, 676100, 917832]","[1293580, 1463197, 1501890, 1000204]",4,"[334254, 355899, 1430584]","[0, 0, 1]",0.838054


In [163]:
train.to_csv("../../data/train_preprocesses.csv")

## Preprocess test

In [164]:
test = pd.read_csv(PATH_TO_TEST, error_bad_lines=False, sep = '[  . ? , !]?\t', 
                   header=None)

/Users/kitashov/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [165]:
test.head()

,0,1,2,3,4,5
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,4,"разумеется , нет ."


In [166]:
test.columns = test_column_names

In [167]:
test.head()

,context_id,context_2,context_1,context_0,reply_id,reply
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а,нет,4,"разумеется , нет ."


In [168]:
test['context_2'] = test['context_2'].apply(lambda x: sentence_to_indices(x))

In [169]:
test['context_1'] = test['context_1'].apply(lambda x: sentence_to_indices(x))

In [170]:
test['context_0'] = test['context_0'].apply(lambda x: sentence_to_indices(x))

In [172]:
test['reply'] = test['reply'].apply(lambda x: sentence_to_indices(x))

In [173]:
test.head()

,context_id,context_2,context_1,context_0,reply_id,reply
0,138920940977,"[1439102, 1463197, 334254, 430126, 1326602, 14...","[743031, 1098780, 1554817]",[1141967],0,"[1858020, 1564319]"
1,138920940977,"[1439102, 1463197, 334254, 430126, 1326602, 14...","[743031, 1098780, 1554817]",[1141967],1,"[1141967, 1463197, 743031, 110812, 1564319]"
2,138920940977,"[1439102, 1463197, 334254, 430126, 1326602, 14...","[743031, 1098780, 1554817]",[1141967],2,"[1141967, 1564319]"
3,138920940977,"[1439102, 1463197, 334254, 430126, 1326602, 14...","[743031, 1098780, 1554817]",[1141967],3,"[1372774, 1463197, 1141967, 1564319]"
4,138920940977,"[1439102, 1463197, 334254, 430126, 1326602, 14...","[743031, 1098780, 1554817]",[1141967],4,"[1411482, 1463197, 1141967, 1564319]"


In [174]:
test.to_csv("../../data/public_preprocessed.csv")